In [40]:
using DynamicPolynomials, SparseArrays, LinearAlgebra

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,2))
f=generate_random_poly(v)


# unit sphere constraint
m=n
q=floor(Int64,n/m)
R=ones(Float64,m)./n
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")


l=ceil(Int64, n/7)


h=Vector{Polynomial{true,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=5
Number of equality constraints: l=1


In [41]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [42]:
k=2
println("Relaxed order: k=",k)
println("====================")
t=1
println("Sparse order: t=",t)

Relaxed order: k=2
Sparse order: t=1


In [43]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val=ctpPOP.POP_TS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=

12
  Size of the largest block: s^max=16
  Number of equality trace constraints: zeta=162
Modeling time:
  0.660364 seconds (568.93 k allocations: 29.195 MiB, 7.73% gc time)
iter=1   val=-3.7135657314248958   gap=1.8843026072145847e-16   feas=3.2912294816992214
iter=2   val=-1.2378552438082986   gap=0.34671249047596725   feas=0.7691505495624673
iter=4   val=-1.3927861305671922   gap=0.18989724054184542   feas=0.5597131337070681
iter=8   val=-0.9435117542087565   gap=0.061592382408651454   feas=0.4290813178822494
iter=16   val=-1.0030447930526174   gap=0.049488685036209464   feas=0.34365229104534595
iter=32   val=-0.997841052273831   gap=0.04091820424833511   feas=0.1799094004071204
iter=64   val=-0.8206767344573591   gap=0.029999113549790818   feas=0.06583127494954316
iter=128   val=-0.7801771241140086   gap=0.03709259071264526   feas=0.024739162376422673
iter=256   val=-0.7657588169649815   gap=0.029194638452097438   feas=0.010846253571401128
iter=512   val=-0.7678479236539957   gap=0

-0.7725239238330983

In [44]:
using TSSOS

@time tssos_first([[f];g;h],x,k,numeq=length(h),TS="block",quotient=false);

------------------------------------------------------
The sizes of blocks:
[16, 5]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 86              
  Cones                  : 0               
  Scalar variables       : 22              
  Matrix variables       : 7               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  

In [45]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       24
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       34

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.0